In [1221]:
select_example = input('Path or Example:')
FILE_PATH = None
SAMPLES = {
    1: 'KOAX_20240521_1844',
    2: 'KOAX_20240521_1849',
    3: 'KOAX_20240523_2138',

}
if select_example.isdigit():
    FILE_PATH = f"{'./Samples/'}{SAMPLES[int(select_example)]}" # I don't care if it is out of range, I just want to test the code
else:
    FILE_PATH = select_example
import os
fsize = os.stat(FILE_PATH).st_size
f = open(FILE_PATH, 'rb')
import bz2
import namedstruct as ns
import logging

In [1222]:
f.seek(0)

log = logging.getLogger(__name__)

VHR = ns.nstruct(
    (ns.char[6],'Tape'),            # Magic number: AR2V00
    (ns.char[2],'Version'),         # Version number
    (ns.char[1],'End of Tape'),     # End of Tape marker
    (ns.char[3],'Sequence Number'), # Extension number
    (ns.uint32,'Date_d'),           # Date of Volume, days since 1/1/1970
    (ns.uint32,'Time_ms'),          # Time of Volume, milliseconds since midnight
    (ns.char[4],'Station'),         # Station ID (ICAO)
    name='VHR',padding=1,endian='>')
LDM_CR = ns.nstruct(
    (ns.int32,'ctrl_word'),      # Control word - lenth of the compressed data
    (ns.raw,'data'),                       # This data is bz2 compressed when saved
    name='LDM_CR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word


L2A = ns.nstruct(
    (VHR,'VHR'),                       # Volume Header Record
    (LDM_CR[0],'records'),             # LDM_CR records 
    name='L2A',padding=1,endian='>')

# test = VHR.parse(f.read(24))
fi, __ = L2A.parse(f.read())
# go back to the first LDM_CR
f.seek(24)
while(f.tell() < fsize): # read each LDM_CR into a record and add it to the archive - we do this since the datatype is raw here.
    l = f.tell()
    s = int.from_bytes(f.read(4))
    if(s==b'\xff\xff'):
        break
    f.seek(l)
    d = f.read(s+4)
    if (d == b''):
        log.debug('bad data')
    ldm = LDM_CR.create(d)
    fi.records.append(ldm)


In [1223]:

if True:
    f.close() # close the file
    log.setLevel(0)

msg_header_channel_enum = ns.enum('msg_header_channel_enum', None, ns.uint8, bitwise=True,
                    # Note: if both redundent channels are 0s, single channel mode. 
                    SINGLE = 0,  # single channel mode
                    RC_1 = 0x1,    # redundant channel 1
                    RC_2 = 0x2,    # redundant channel 2
                    unused = 0x4,  # unused. 
                    ORDA = 0x8     # ORDA channel (vs legacy)
                    )
# Message header structure 
msg_header = ns.nstruct(
        (ns.uint32[3],'prepad'),                # padding
        (ns.uint16,'message_size'),             # size of message in halfwords
        (msg_header_channel_enum,'channel'),    # actually bit field
        (ns.uint8,'message_type'),              # type of message
        (ns.uint16,'sequence'),                 # sequence number of message
        (ns.uint16,'date'),                     # days since 1/1/1970
        (ns.uint32,'milliseconds'),             # milliseconds since midnight
        # Remember, is these 2 are not 1, then the message is extended and needs to be combined with the next message
        (ns.uint16,'segments'),                 # number of segments in message  
        (ns.uint16,'segment'),                  # segment number of this message
        name='msg_header', padding=1, prepack=ns.packvalue(2432,'message_size'), endian='>') # TODO: Use size of message to determine size of message 

# Base message structure
msg_base = ns.nstruct(
    (msg_header,'header'),
    (ns.raw,'data'),
    name='msg_base',padding=1,endian='>')

# UNCOMPRESSED RECORD data
LDM_UR = ns.nstruct(
    (msg_base[0],'msg'),                # Messages - not sure that this will work since it will hold extended data
    name='LDM_UR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word

RECORD_HOLDER = ns.nstruct(
    (LDM_UR[0],'ldm'),
    name='RECORD_HOLDER',padding=1,endian='>')




In [1224]:
# BASE TYPES
CODE_1 = ns.enum('CODE_1',None, ns.uint8, bitwise=False)
CODE_2 = ns.enum('CODE_1',None, ns.uint16, bitwise=False)
SI_1 = ns.nstruct((ns.uint8,'si'),name='SI_1',padding=1,endian='>')
SI_2 = ns.nstruct((ns.uint16,'si'),name='SI_2',padding=1,endian='>')
SI_4 = ns.nstruct((ns.uint32,'si'),name='SI_4',padding=1,endian='>')
SSI_2 = ns.nstruct((ns.int16,'ssi'),name='sSI_2',padding=1,endian='>')
SSI_4 = ns.nstruct((ns.int16,'ssi'),name='sSI_4',padding=1,endian='>')

In [1225]:
# MSG 31 TYPES

msg_31_table = ns.nstruct(
    (ns.raw,'raw_data'), # raw data
    name='msg_31_table',padding=1,endian='>')

db_hdr = ns.nstruct(      # 33 - MSG 31 - Data Block Header
    (ns.char,'db_type'),
    (ns.char[3],'dm_type'),
    (ns.uint16,'db_size'), # size of the data block (bytes) / reserved
    name='db_hdr',padding=1,endian='>')

t_17a = ns.nstruct(      # 17A - MSG 31 - Data Header Block
    (ns.char[4],'ICAO'), # ICAO of the station
    (ns.uint32,'time_ms'), # time of the observation
    (ns.uint16,'date_d'), # date of the observation
    (ns.uint16,'az_n'), # azimuth # of the observation
    (ns.single,'az_a'), # azimuth angle of the observation (32 bit)
    (CODE_1,'compression'), # compression type, values can be 0-3
    (ns.uint8,'spare_17'), # spare
    (ns.uint16,'rad_len'), # length of the radial data (uncompressed) ubckydibg the header, 9360-14296
    (CODE_1,'az_res_spacing'), # azimuth resolution spacing, 1: 0.5, 2: 1.0
    (CODE_1, 'rad_stat'), # radial status 0-132
    (ns.uint8,'el_n'), # elevation number
    (ns.single,'el_a'), # elevation angle
    (CODE_1, 'rad_spot_blacking'), # radial spot blanking: 0=none, 1=radical, 2=elevation, 4=volume
    (SI_1,'az_index_mode'), #0: none, 1-100-> 0.001 to 1.00 deg
    (ns.uint16,'data_block_count'), # number of data blocks
    (ns.uint32,'dbp_vol_dc'), # data block pointer volume data constant     17E
    (ns.uint32,'dbp_el_dc'), # data block pointer elevation data constant   17F
    (ns.uint32,'dbp_rad_dc'), # data block pointer radial data constant     17H
    (ns.uint32,'dbp_ref_m'), # data block pointer for REF moments       17B, 17I
    (ns.uint32,'dbp_vel_m'), # data block pointer for VEL moments       17B, 17I
    (ns.uint32,'dbp_sw_m'), # data block pointer for SW moments         17B, 17I
    (ns.uint32,'dbp_zdr_m'), # data block pointer for ZDR moments       17B, 17I
    (ns.uint32,'dbp_phi_m'), # data block pointer for PHI moments       17B, 17I
    (ns.uint32,'dbp_rho_m'), # data block pointer for RHO moments       17B, 17I
    (ns.uint32,'dbp_cfp_m'), # data block pointer for CFP moments       17B, 17I
    name='t_17a',padding=1,endian='>')

t_17e = ns.nstruct(      # 17E - MSG 31 - Volume Data Constants
    (db_hdr,'header'),
    (ns.uint8,'v_major'), 
    (ns.uint8,'v_minor'), 
    (ns.single,'lat'), # latitude of the radar
    (ns.single,'lon'), # longitude of the radar
    (SSI_2,'height'), # height of the radar
    (ns.uint16,'fh_h'), # height of the feedhorn
    (ns.single,'r_cal_c'), # calibration constant
    (ns.single,'shv_h_tx'), # horizontal tx power
    (ns.single,'shv_v_tx'), # vertical tx power
    (ns.single,'sdr_cal'), # calibration of zdr
    (ns.single,'dp_i'), # inital dp 
    (ns.uint16,'vcp'), # volume coverage pattern
    (CODE_2,'proc_stat_28'), # processing status options - says uint16, but it is bitwise so
    (ns.uint16,'zdr_bewm'), # zdr bias estimate weighted mean
    (ns.uint8[6],'spare_padding'), # padding - 46-51
    name='t_17e',padding=1,endian='>',base=msg_31_table)

t_17f = ns.nstruct(      # 17F - MSG 31- Elevation Data Constants
    (db_hdr,'header'),
    (SSI_2,'atmos'), # atmospheric attenuation
    (ns.single,'cal_z'), # calibration constant - scaling for signal processor
    name='t_17f',padding=1,endian='>',base=msg_31_table)

t_17h = ns.nstruct(      # 17H - MSG 31 - Radical Data Constants
    (db_hdr,'header'),
    (SI_2,'unabm_r_int'), # unambiguous range interval size
    (ns.single,'noise_h'), # noise level for horizontal
    (ns.single,'noise_v'), # noise level for vertical
    (SI_2, 'nyq'), # nyquist velocity
    (ns.uint16,'rad_flags'), # radial flags
    (ns.single,'cal_const_h'), # calibration constant for horizontal
    (ns.single,'cal_const_v'), # calibration constant for vertical
    name='t_17h',padding=1,endian='>',base=msg_31_table)

t_17i = ns.nstruct(      # 17I - TODO
    (ns.raw,'data'), # data
    name='t_17i',padding=1,endian='>', base=msg_31_table)

t_17i_ref = ns.nstruct(      # 17I - MSG 31 - REF Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_ref',padding=1,endian='>')

t_17i_vel = ns.nstruct(      # 17I - MSG 31 - VEL Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_vel',padding=1,endian='>')

t_17i_sw = ns.nstruct(      # 17I - MSG 31 - SW Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_sw',padding=1,endian='>')

t_17i_zdr = ns.nstruct(      # 17I - MSG 31 - ZDR Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_zdr',padding=1,endian='>')

t_17i_phi = ns.nstruct(      # 17I - MSG 31 - DP Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_phi',padding=1,endian='>')

t_17i_rho = ns.nstruct(      # 17I - MSG 31 - CC Moments
    (ns.raw,'data'), # datarho
    base=t_17i,name='t_17i_rho',padding=1,endian='>')

t_17i_cfp = ns.nstruct(      # 17I - MSG 31 - Clutter Filter Pwoeer Removed - Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_cfp',padding=1,endian='>')

t_17b = ns.nstruct(      # 17B - MSG 31 - Data Block
    (db_hdr,'header'),
    (ns.uint8,'reserved_a'), 
    (ns.uint8,'reserved_b'), 
    (ns.uint16,'n_gates'), # number of gates
    (SI_2,'gate_range'), # gate range to center of first gate - 0.000 to 32.768 km
    (SI_2,'gate_samp_int'), # gate sample interval - 0.25 to 4.0 km
    (SI_2,'tover'), # threshold value for similar to resolution, 0.0 to 20.0 dB
    (SI_2,'SNR_thres'), # signal to noise threshold for valid data, -12.0 to 20.0 dB
    (CODE_1,'ctrl_flags'), # control flags for recombined - 0=none, 1=az rads, 2=range gates, 3=rads and gates to legacy res
    (ns.uint8,'dw_size'), # data word size, 8 or 16
    (ns.single,'scale'), # scale value for data momens int->float
    (ns.single,'offset'), # offset value for data moments int->float
    (t_17i,'data moments'), # data moments                            17I       
    name='t_17b',padding=1,endian='>',base=msg_31_table)

msg_31_raw = ns.nstruct(       # 17A
    (msg_header,'header'),
    (t_17a,'header_31'), 
    (msg_31_table,'data_RAW'), # This array causes issues since it is variable length and the data is also variable length. Making it a single element for now.
    name='msg_31_raw',padding=1,endian='>')

msg_31 = ns.nstruct(       # 17A
    (msg_header,'header'),
    (t_17a,'header_31'), 
    (msg_31_table[0],'data'), # This array causes issues since it is variable length and the data is also variable length. Making it a single element for now.
    name='msg_31',padding=1,endian='>')


In [1226]:
# MSG 15 TYPES
t_14a = ns.nstruct(      # 14A - Clutter Filter Map Header  
    (ns.uint16,'date_d'), # days since 1/1/1970
    (ns.uint16,'time_m'), # minutes since midnight
    (ns.uint16,'el_n'), # number of elevations
    name='t_14a',padding=1,endian='>')

t_14_r = ns.nstruct(      # 14A - Clutter Filter Map Range
    (CODE_2,'op_code'), # operation code
    (ns.uint16,'stop_range'), # stop range
    name='t_14_r',padding=1,endian='>')

t_14_az = ns.nstruct(      # 14A - Clutter Filter Map Azimuth
    (ns.uint16,'rz_n'),  # number of range bins in the azimuth
    (t_14_r[0],'ranges'), # ranges
    name='t_14_az',padding=1,endian='>')

t_14_el = ns.nstruct(      # 14A - Clutter Filter Map Elevation
    (t_14_az[0],'azimuths'), # azimuths
    name='t_14_el',padding=1,endian='>')

msg_15 = ns.nstruct(
    (msg_header,'header'),
    (t_14a,'header_15'),
    name='msg_15',padding=1,endian='>')

In [1227]:
# message type 2 items

t4_rda_stat_enum = ns.enum('rda_stat_enum',None, CODE_2, bitwise=True,
                        Startup = 0x2, # Startup
                        Standby = 0x4, # Standby
                        Restart = 0x8, # Restart
                        Operate = 0x10, # Operate
                        SpareA = 0x20, # Spare
                        SpareB = 0x40 # Shutdown
)

t4_op_stat_enum = ns.enum('op_stat_enum',None, CODE_2, bitwise=True,
                            Online = 0x2, # Online
                            MaintActReq = 0x4, # Maintenance action required
                            MaintActMand = 0x8, # Maintenance action mandatory
                            CmdShtdwn = 0x10, # Commanded shutdown
                            Inop = 0x20 # Inoperable
)

t4_ctrl_stat_enum = ns.enum('ctrl_stat_enum',None, CODE_2, bitwise=True,
                            LocalOnly = 0x2, # Local control
                            RemoteOnly = 0x4, # Remote control
                            LocRem = 0x8, # Local or remote control
)
t4_aux_pwr_gen_stat_enum = ns.enum('aux_pwr_gen_stat_enum',None, CODE_2, bitwise=True,
                            OnAux = 0x1, # On auxiliary power
                            UtilAvail = 0x2, # Utility power available
                            GenOn = 0x4, # Generator on
                            TfSwitch = 0x8, # Transfer switch - manual
                            CmdSwo = 0x10, # Commanded switch over
)

t4_dat_tx_en_enum = ns.enum('dat_tx_en_enum',None, CODE_2, bitwise=True,
                            Non = 0x2, # None
                            Ref = 0x4, # Reflectivity
                            Vel = 0x8, # Velocity
                            Sw = 0x10, # Spectrum width
)
t4_rda_ctrl_auth_enum = ns.enum('rda_ctrl_auth_enum',None, CODE_2, bitwise=True,
                            NoAct = 0x0, # No action
                            LocalReq = 0x2, # Local control request
                            RemoteReq = 0x4, # Remote control request - local released
)
t4_op_mode_enum = ns.enum('op_mode_enum',None, CODE_2, bitwise=True, Oper = 0x4, Maint = 0x8,)
t4_super_res_stat_enum = ns.enum('super_res_stat_enum',None, CODE_2, bitwise=True,Enabled = 0x2, Disabled = 0x4)
t4_clutter_mit_d_stat_enum = ns.enum('clutter_mit_d_stat_enum',None, CODE_2, bitwise=True,
                                        Disabled = 0x0,
                                        Enabled = 0x1,
                                        SegA = 0x2,
                                        SegB = 0x4,
                                        SegC = 0x8,
                                        SegD = 0x10,
                                        SegE = 0x20)
t4_rda_scan_dat_flag_enum = ns.enum('rda_scan_dat_flag_enum',None, CODE_2, bitwise=True,
                                    AVSET_EN = 0x1,
                                    AVSET_DIS = 0x2,
                                    EBC_EN = 0x4,
                                    RDA_LOG_EN = 0x8,
                                    TIME_SERIES_DAT_REC = 0x10)
t4_rda_alarms_enum = ns.enum('rda_alarms_enum',None, CODE_2, bitwise=True,
                            NoAlarm = 0,
                            TowerUtil = 0x1,
                            Pedestal = 0x2,
                            Tx = 0x4,
                            Rx = 0x8,
                            RDACtrl = 0x10,
                            Comm = 0x20,
                            SigProc = 0x40
)

t4_cmd_ack_enum = ns.enum('cmd_ack_enum',None, CODE_2, bitwise=False,NoAck = 0,RemVCPRec=1,ClutterBypRec=2,ClutterCensorRec=3,RedundChanCmdAcc=4)

t4_chan_ctrl_enum = ns.enum('chan_ctrl_enum',None, CODE_2, bitwise=False,Controlling=0,NonControlling=1)
t4_spot_blank_enum = ns.enum('spot_blank_enum',None, CODE_2, bitwise=True,NotInstalled=0x0,Enabled=0x2,Disabled=0x4)
t4_tps_stat_enum = ns.enum('tps_stat_enum',None, CODE_2, bitwise=False,NotInstalled=0,Off=1,OK=3,Unknown=4)
t4_rms_ctrl_stat_enum = ns.enum('rms_ctrl_stat_enum',None, CODE_2, bitwise=False,NonRMS=0,RMSCtrl=2,RDACtrl=4)
t4_performance_stat_enum = ns.enum('performance_stat_enum',None, CODE_2, bitwise=False,NoCmd=0,ForcePerfPend=1,IP=2)
t4_sig_proc_opt_enum = ns.enum('sig_proc_opt_enum',None, CODE_2, bitwise=False,Enabled=1)
msg_2 = ns.nstruct(      # 4 -- messgae type 2
    (msg_header,'header'),
    (t4_rda_stat_enum,'rda_stat'), # RDA status
    (t4_op_stat_enum,'op_state'), # operability status
    (t4_ctrl_stat_enum,'ctrl_state'), # control status
    (t4_aux_pwr_gen_stat_enum,'aux_pwr_gen_state'), # auxiliary power generator status
    (ns.uint16,'avg_tx_pwr'), # average transmit power
    (SI_2,'ref_calib'), # reflectivity calibration correction
    (t4_dat_tx_en_enum,'dat_tx_en'), # data transmission enabled
    (ns.int16,'vcp'), # volume coverage pattern - 0 = none, pattern sel:  neg = rda local, pos = rda remote
    (t4_rda_ctrl_auth_enum,'rda_ctrl_auth'), # RDA control authority
    (SI_2,'rda_build'), # RDA build number
    (t4_op_mode_enum,'op_mode'), # operation mode
    (t4_super_res_stat_enum,'super_res_stat'), # super resolution status
    (t4_clutter_mit_d_stat_enum,'clutter_mit_d_stat'), # clutter mitigation decision status
    (t4_rda_scan_dat_flag_enum,'rda_scan_dat_flag'), # RDA scan data flag TODO: FIX
    (t4_rda_alarms_enum,'rda_alarms'), # RDA alarms
    (t4_cmd_ack_enum,'cmd_ack'), # command acknowledgement
    (t4_chan_ctrl_enum,'chan_ctrl'), # channel control
    (t4_spot_blank_enum,'spot_blank'), # spot blanking
    (ns.uint16,'bypass_map_date_d'), # bypass map generation date - days since 1/1/1970
    (ns.uint16,'bypass_map_time_m'), # bypass map generation time - minutes since midnight
    (ns.uint16,'clutter_map_date_d'), # clutter map generation date - days since 1/1/1970
    (ns.uint16,'clutter_map_time_m'), # clutter map generation time - minutes since midnight
    (SI_2,'vert_ref_cal_corr'), # vertical calibration correction
    (t4_tps_stat_enum,'tps_stat'), # transition power source status
    (t4_rms_ctrl_stat_enum,'rms_ctrl_stat'), # rms control status
    (t4_performance_stat_enum,'performance_stat'), # performance data collection status
    (ns.uint16[14],'alarm_codes'), # table 4A - 800 possible, 17 at a time
    (t4_sig_proc_opt_enum,'sig_proc_options'), # signal processor options
    (ns.uint16[18],'spares'), # spares
    (ns.int16,'stat_version'), # status version number
    (ns.raw,'padding'), # data
    name='msg_2',padding=1,endian='>')

In [1228]:
# MSG 18 TYPES - TODO

# . RDA Adaptation Data


t15_rda_adapt_data_1 = ns.nstruct(      # 15 - RDA Adaptation Data
    (ns.char[12],'adap_f_name'), # adaptation data file name
    (ns.char[4],'adap_f_format'), # adaptation data file format
    (ns.char[4],'adap_f_rev'), # adaptation data file revision
    (ns.char[12],'adap_f_date'), # adaptation data file date
    (ns.char[12],'adap_f_time'), # adaptation data file time
    (ns.single,'lower_pre_limit_sw_ang'), # lower pre-limit switch angle
    (ns.single,'az_enc_latency'), # azimuth encoder latency
    (ns.single,'upper_pre_limit_sw_ang'), # upper pre-limit switch angle
    (ns.single,'el_enc_latency'), # elevation encoder latency
    (ns.single,'az_park'), # azimuth park position
    (ns.single,'el_park'), # elevation park position
    (ns.single[11],'gen_fuel_height_conv'), # generator fuel tank height conversion percents 0-100-10
    (ns.single,'a_min_shelter_temp'), # minimum shelter temperature for alarm
    (ns.single,'a_max_shelter_temp'), # maximum shelter temperature for alarm
    (ns.single,'a_min_shelter_ac_temp_diff'), # minimum shelter air conditioner difference for alarm
    (ns.single,'a_max_tx_air_temp'), # maximum transmitter air temperature for alarm
    (ns.single,'a_max_radome_temp'), # maximum radome temperature for alarm
    (ns.single,'a_max_radome_temp_rise'), # maximum radome temperature difference for alarm
    (ns.single,'lower_dead_limit_sw_ang'), # lower dead limit switch angle
    (ns.single,'upper_dead_limit_sw_ang'), # upper dead limit switch angle
    (ns.uint8[4],'spares_144'), # spares
    (ns.single,'a_min_gen_rm_temp'), # minimum generator room temperature for alarm
    (ns.single,'a_max_gen_rm_temp'), # maximum generator room temperature for alarm
    (ns.single,'spip_5v_reg_lim'), # tollerance for 5v reg
    (ns.single,'spip_15v_reg_lim'), # tollerance for 15v reg
    (ns.uint8[12],'spares_164'), # spares
    (ns.char[4],'rpg_co_loc'), # RPG co-location
    (ns.char[4],'spectrum_filter_installed'), # spectrum filter installed
    (ns.char[4],'tps_installed'), # tps installed
    (ns.char[4],'rms_installed'), # rms installed
    (ns.uint32,'hvdl_test_interval_h'), # perormance test interval
    (ns.uint32,'rpg_lt_test_interval_m'), # RPG LoopTest interval
    (ns.uint32,'min_stable_util_time_m'), # minimum stable utility power time
    (ns.uint32,'max_gen_auto_exer_time_h'), # maximum generator auto exercise time
    (ns.uint32,'recommend_sw_to_util_interval_m'), # recommended switch to utility interval
    (ns.single,'low_fuel_warning'), # low fuel warning
    (ns.uint32,'cfg_chan_num'), # configured channel number
    (ns.uint8[4],'spares_220'), # TODO: FIX HERE
    (ns.uint32,'redun_chan_cfg'), # redundant channel configuration
    (ns.single[104],'test_sig_atten_loss_db'), # test signal attenuation loss at [location] dB
    (ns.uint8[24],'sppares_644'),
    (ns.single,'path_loss_7'), # paath loss - vertical is heliax to 4 at 16
    (ns.uint8[48],'missing'), # TODO: missing - temp to figure out how much im missing
    (ns.uint8[12],'spares_672'), # spares
    (ns.single,'path_loss_13'), # path loss, 2a9a9 rf delay line
    (ns.uint8[16],'spares_696'), # spares
    (ns.single,'path_loss_28'), # path loss, horizontal if heliax to 4 at 17
    (ns.single,'h_coup_tx_loss'), # horizontal coupler tx loss
    (ns.uint8[8],'spares_760'), # spares
    (ns.single,'path_loss_32'), # path loss - wg02 harmonic filter
    (ns.single,'path_loss_33'), # path loss - waveguide klystron to switch
    (ns.uint8[4],'spares_776'), # spare
    (ns.single,'path_loss_35'), # path loss - wg06 spectrum filter
    (ns.uint8[12],'spares_784'), # spares
    (ns.single,'path_loss_39'), # path loss - wg04 circulator
    (ns.single,'path_loss_40'), # path loss - a6 arc detector
    (ns.uint8[4],'spares_804'), # spare
    (ns.single,'path_loss_42'), # path loss - 1dc1 tx coupler coupling
    (ns.single,'path_loss_43'), # path loss - a33 pad
    (ns.single,'path_loss_44'), # path loss - coax tx rf to a33 pad
    (ns.single,'path_loss_45'), # path loss - a20j1_4 power splitter
    (ns.single,'path_loss_46'), # path loss - a20j1_3 power splitter
    (ns.single,'path_loss_47'), # path loss - a20j1_2 power splitter
    (ns.single,'h_coupler_cw_loss'), # horizontal coupler cw loss
    (ns.single,'v_coupller_tx_loss'), # vertical coupler tx loss
    (ns.uint8[4],'spare_840'), # spare
    (ns.single,'ame_ts_bias'), # ame test signal bias
    (ns.single,'path_loss_52'), # path loss - 1at4 tx coupler pad
    (ns.single,'v_coup_cw_loss'), # vertical coupler cw loss
    (ns.single[2],'spares_856'), # spare
    (ns.single,'pwr_sense_bias'), # power sense bias
    (ns.single,'ame_v_noise_enr'), # ame noise source excess noise ratio
    (ns.single,'path_loss_58'), # path loss - 4 at 17 attenuator
    (ns.single,'path_loss_59'), # path loss - ifdr if anti-alias filter
    (ns.single,'path_loss_60'), # path loss - a20j1_5 power splitter
    (ns.single,'path_loss_61'), # path loss - at5 50db attenuator
    (ns.uint8[4],'spare_888'),# path loss 62?
    (ns.single,'path_loss_63'), # path loss - a39 rf_if burst mixer
    (ns.single,'path_loss_64'), # path loss - ar1 burst if amplifier
    (ns.single,'path_loss_65'), # path loss - ifdr burst anti-alias filter
    (ns.single,'path_loss_66'), # path loss - dc3 j1_3 6db coupler through
    (ns.single,'path_loss_67'), # path loss - 4dc3 j1 to 4a39 l
    (ns.single,'path_loss_68'), # path loss - at2 + at3 26db coho attenuator
    (ns.uint8[4],'spare_916'), # path loss 69? 
    (ns.single,'chan_cal_diff'), # channel calibration difference
    (ns.single,'path_loss_70_71'), # path loss - spares
    (ns.uint8[8],'spares_928'), # spares
    (ns.single,'v_ts_cw_bias'), # ame vertical test sig power
    (ns.single[13],'H_RNScale'), # horizontal receiver noice normaliztion (deg ranges)
    (ns.single[13],'atmos_loss'), # atmospheric loss 0 2 way per km, (deg ranges)
    (ns.single[12],'el_index_bpm'), # elevation angle for bypass map generation
    (ns.uint32,'tx_freq_mhz'), # transmitter frequency
    (ns.single,'base_data_tcn'), # point clutter suppression threshold
    (ns.single,'ref_data_tover'), # reflectivity data overlay threshold
    (ns.single,'tar_h_dbz0_lp'), # horizontal targer sys calibration for long pulse
    (ns.single,'tar_v_dbz0_lp'), # horizontal target sys calibration for short pulse
    (ns.uint32,'init_phi_dp'), # initial phi dp
    (ns.uint32,'norm_init_phi_dp'), # normalized initial phi dp
    (ns.single,'lx_lp'), # matched filter loss for long pulse
    (ns.single,'lx_sp'), # matched filter loss for short pulse
    (ns.single,'hydro_refract_factor'), # hydro-meteor refractive factor
    (ns.uint8[4],'spares_1132'), # spares
    (ns.single,'ant_gain_rad'), # antenna gain including radome
    (ns.uint8[12],'spares_1140'),
    (ns.single,'vel_degrade_limit'), # velocity degradation limit
    (ns.single,'sw_degrade_limit'), # spectrum width degradation limit
    (ns.single,'h_noisetemp_degrade_limit'), # horizontal noise temperature degradation limit
    (ns.uint32,'h_min_noisetemp'), # horizontal minimum noise temperature
    (ns.single,'v_noisetemp_degrade_limit'), # vertical noise temperature degradation limit
    (ns.uint32,'v_min_noisetemp'), # vertical minimum noise temperature
    (ns.single,'kly_degrade_limit'), # klystron degradation limit
    (ns.single,'ts_coho'), # coho power at a1j4
    (ns.single,'h_ts_cw'), # ame horizontal test signal power
    (ns.uint8[8],'spares_1188'), # spares
    (ns.single,'ts_stalo'), # stalo power at a1j2
    (ns.single,'ame_h_noise_enr'), # ame horizontal noise source excess noise ratio
    (ns.single,'tx_peak_pwr_high_lim'), # transmitter peak power high limit
    (ns.single,'tx_peak_pwr_low_lim'), # transmitter peak power low limit
    (ns.single,'h_dbz0_delta_limit'), # horizontal cbz0 limit - computed vs target diff
    (ns.single,'bpm_gen_noise_thr'), # bypass map generation noise threshold
    (ns.single,'bpm_gen_reject_ratio_thr'), # bypass map generation reject ratio threshold
    (ns.single,'clutter_suppression_degreade_limit'), # clutter suppression degradation limit
    (ns.uint8[4],'spares_1228'), # spares
    (ns.single,'range0_value'), # true valie at start of first range bin
    (ns.single,'tx_pwr_mtr_scale'), # scale factor for tx power byte -> watts
    (ns.single,'v_dbz0_delta_lim'), # diff between computed and target vertical dbz0 limit
    (ns.single,'tar_h_dbz0_sp'), # horizontal target sys calibration for short pulse
    (ns.single,'tar_v_dbz0_sp'), # vertical target sys calibration for short pulse
    (ns.uint32,'delta_prf'), # delta prf
    (ns.int8[8],'spares_1256'), # spares 
    (ns.uint32,'tau_sp'), # pulse width of tx out in sp
    (ns.uint32,'tau_lp'), # pulse width of tx out in lp
    (ns.uint32,'nc_dead_value'), # num of 1/4 km bins corrupt at end of sweep
    (ns.uint32,'tau_rf_sp'), # rf drive pulse width in short pulse
    (ns.uint32,'tau_rf_lp'), # rf drive pulse width in long pulse
    (ns.single,'seg1lim'), # clutter map boundry elev between 1 and 2 segments
    (ns.single,'site_lat_sec'), # site latitude  seconds
    (ns.single,'site_lon_sec'), # site longitude  seconds
    (ns.uint8[4],'spares_1296'), # spares
    (ns.uint32,'site_lat_deg'), # site latitude  degrees
    (ns.uint32,'site_lat_min'), # site latitude  minutes
    (ns.uint32,'site_lon_deg'), # site longitude  degrees
    (ns.uint32,'site_lon_min'), # site longitude  minutes
    (ns.char[4],'site_lat_dir'), # site latitude  direction
    (ns.char[4],'site_lon_dir'), # site longitude  direction
    (ns.uint8[4],'spares_1324'), # spares
    (ns.uint8[1064],'spares_1328_a'), # spares # full size is 1172
    name='t15_rda_adapt_data_1',padding=1,endian='>')

t15_rda_adapt_data_2 = ns.nstruct(      # 15 - RDA Adaptation Data - segment 2
    (ns.uint8[108],'spares_1328_b'), # spares
    (ns.uint8[1172],'spares_2500'), # spares
    (ns.uint8[1112],'spares_3672_a'), # spares
    name='t15_rda_adapt_data_2',padding=1,endian='>'
)

t15_rda_adapt_data_3 = ns.nstruct(      # 15 - RDA Adaptation Data - segment 3
    (ns.uint8[60],'spares_3672_b'), # spares
    (ns.uint8[1172],'spares_3672'), # spares
    (ns.uint8[1160],'spares_4844_a'), # spares
    name = 't15_rda_adapt_data_3',padding=1,endian='>')

t15_rda_adapt_data_4 = ns.nstruct(      # 15 - RDA Adaptation Data - segment 4
    (ns.uint8[12],'spares_4844_b'), # spares
    (ns.uint8[1148],'spares_s'), # spares
    # (ns.uint8[1172],'spares_6016'), # spares
    # (ns.uint8[1172],'spares_7188'), # spares
    (ns.single,'az_corr_factor'), # azimuth correction factor
    (ns.single,'el_corr_factor'), # elevation correction factor
    (ns.char[4],'site_name'), # site name # 1196
    (ns.raw,'padding'), # padding
    name='t15_rda_adapt_data_4',padding=1,endian='>')

t15_rda_adapt_data_5 = ns.nstruct(      # 15 - RDA Adaptation Data - segment 5
    (ns.uint8[1172],'spares_6016'), # spares
    (ns.uint8[1172],'spares_7188'), # spares
    name='t15_rda_adapt_data_5',padding=1,endian='>')

msg18_1 = ns.nstruct(      # 18 - RDA Adaptation Data
    (msg_header,'header'),
    (t15_rda_adapt_data_1,'data'), # adaptation data
    name='msg18_1',padding=1,endian='>')

msg18_2 = ns.nstruct(      # 18 - RDA Adaptation Data
    (msg_header,'header'),
    (t15_rda_adapt_data_2,'data'), # adaptation data
    name='msg18_2',padding=1,endian='>')

msg18_3 = ns.nstruct(      # 18 - RDA Adaptation Data
    (msg_header,'header'),
    (t15_rda_adapt_data_3,'data'), # adaptation data
    name='msg18_3',padding=1,endian='>')

msg18_4 = ns.nstruct(      # 18 - RDA Adaptation Data
    (msg_header,'header'),
    (t15_rda_adapt_data_4,'data'), # adaptation data
    name='msg18_4',padding=1,endian='>')

msg18_5 = ns.nstruct(      # 18 - RDA Adaptation Data
    (msg_header,'header'),
    (t15_rda_adapt_data_5,'data'), # adaptation data
    name='msg18_5',padding=1,endian='>')

In [1229]:
# Message 3 types - TODO
# Performance/Maintenance Data 
# tons of metadata

t5_comms = ns.nstruct(      # 5 - Message Type 3 - Communications
    (ns.uint8,'spare_1'), # reserved
    (ns.uint16,'loopback_test_stat'), # loopback test status
    (ns.uint32,'t1_out_frames'), # T1 output frames
    (ns.uint32,'t1_in_frames'), # T1 input frames
    (ns.uint32,'router_mem_used'),
    (ns.uint32,'router_mem_free'),
    (ns.uint16,'router_mem_util'),
    (ns.uint16,'rpg_route'),
    (ns.uint8[8],'spare_13'), # reserved
    (ns.uint32,'csu_24h_err_s'),
    (ns.uint32,'csu_24h_sev_err_s'),
    (ns.uint32,'csu_24h_sev_err_framing_s'),
    (ns.uint32,'csu_24h_unavil_s'),
    (ns.uint32,'csu_24h_ctrl_slip_s'),
    (ns.uint32,'csu_24h_path_code_viol_n'),
    (ns.uint32,'csu_24h_line_code_viol_s'),
    (ns.uint32,'csu_24h_bursty_err_s'),
    (ns.uint32,'csu_degraded_m'),
    (ns.uint8[2],'spare_39'), # reserved
    (ns.uint32,'lan_sw_cpu_util'),
    (ns.uint32,'lan_sw_mem_util'),
    (ns.uint8,'spare_44'),
    (ns.uint16,'ifdr_chassis_temp'),
    (ns.uint16,'ifdr_fpga_temp'),
    (ns.uint16,'ntp_sync_stat'),
    (ns.uint8[5],'spare_48'), # reserved
    (ns.uint16,'ipc_stat'),
    (ns.uint16,'cmd_chan_ctrl'),
    (ns.uint8[3],'spare_55'), # reserved
    name='t5_comms',padding=1,endian='>')

t5_ames = ns.nstruct(      # 5 - Message Type 5 - AME
    
"""
    Polarization - int16
    AME Internal Temperature - real4
    AMEReceiverModuleTemperature - real4
    AMEBITE/CALModuleTemperature - real4
    AME Peltier Pulse Width Modulation - int16
    AME Peltier Status - int16
    AME A/D Converter Status - int16
    ame state - int16
    AME +3.3V PS Voltage - real4
    AME +5V PS Voltage - real4
    AME +6.5V PS Voltage - real4
    AME +15V PS Voltage - real4
    AME +48V PS Voltage - real4
    AME STALO Power - real4
    peltier current - real4
    adc calib ref voltage - real4
    ame mode - int16
    ame peliterer mode - int16
    ame pelt in fan curr - real4
    ame pelt out fan curr - real4
    horiz tr limiter volt - real4
    vert tr limiter volt - real4
    adc calib offser volt - real4
    adc calib gain corr - real4

""")

# a bunch more stuff...

c:\Python312\Lib\site-packages\namedstruct\namedstruct.py:2193: StructDefWarning: A struct is not named: ('\n    Polarization - int16\n    AME Internal Temperature - real4\n    AMEReceiverModuleTemperature - real4\n    AMEBITE/CALModuleTemperature - real4\n    AME Peltier Pulse Width Modulation - int16\n    AME Peltier Status - int16\n    AME A/D Converter Status - int16\n    ame state - int16\n    AME +3.3V PS Voltage - real4\n    AME +5V PS Voltage - real4\n    AME +6.5V PS Voltage - real4\n    AME +15V PS Voltage - real4\n    AME +48V PS Voltage - real4\n    AME STALO Power - real4\n    peltier current - real4\n    adc calib ref voltage - real4\n    ame mode - int16\n    ame peliterer mode - int16\n    ame pelt in fan curr - real4\n    ame pelt out fan curr - real4\n    horiz tr limiter volt - real4\n    vert tr limiter volt - real4\n    adc calib offser volt - real4\n    adc calib gain corr - real4\n\n',)
  warnings.warn(StructDefWarning('A struct is not named: %r' % (members,)))
c

In [1230]:
t11_sector = ns.nstruct(
    (CODE_2,'edge_angle'), # edge angle
    (ns.uint16,'dop_prf_num'), # doppler prf number
    (ns.uint16,'dop_prf_pulse_count'), # doppler prf pulse count per radial
    name='t11_sector',padding=1,endian='>')

t11_elev = ns.nstruct(
    (CODE_2,'elev_angle'), # elevation angle
    (CODE_1,'channel_config'), # channel configuration
    (CODE_1,'wave_form_type'), # wave form type
    (CODE_1,'super_res_ctrl'), # super resolution control
    (ns.uint8,'surv_prf_num'), # surveillance prf number
    (ns.uint16,'surv_prf_pulse_count'), # surveillance prf pulse count per radial
    (CODE_2,'az_rate'), # azimuth rate
    (SSI_2,'reflect_snr_thresh'), # reflectivity snr threshold
    (SSI_2,'vel_snr_thresh'), # velocity snr threshold
    (SSI_2,'sw_snr_thresh'), # spectrum width snr threshold
    (SSI_2,'zdr_snr_thresh'), # zdr snr threshold
    (SSI_2,'phidp_snr_thresh'), # phidp snr threshold
    (SSI_2,'rho_snr_thresh'), # rho snr threshold
    (SSI_2,'cc_snr_thresh'), # cc snr threshold
    (t11_sector,'sector_1'), # sector 1
    (CODE_2,'sup_data'), # supplemental data
    (t11_sector,'sector_2'), # sector 2
    (CODE_2,'ebc_angle'), # ebc angle
    (t11_sector,'sector_3'), # sector 3
    (ns.uint8,'reserved'), # reserved
    name='t11_elev',padding=1,endian='>')

t11_hdr = ns.nstruct(
    (ns.uint16,'msg_size'), # message size in halfwords
    (CODE_2,'pattern_type'), # pattern type - constant elevation cut
    (ns.uint16,'pattern_num'), # pattern number - append c, oper or constant elevation
    (ns.uint16,'num_elev_cuts'), # number of elevation cuts
    (ns.uint8,'vcp_version'), # vcp version number
    (ns.uint8,'clutter_map_group_num'), # clutter map group number
    (CODE_1,'dop_vel_res'), # doppler velocity resolution - 0.5/1.0
    (CODE_1,'pulse_width'), # pulse width - long/short
    (ns.uint8[2],'reserved'), # reserved
    (CODE_2,'vcp_seq'), # vcp sequencing - elevs, max sails cuts, seq active, truncated vcp
    (CODE_2,'vcp_sup_data'), # vcp supplemental data - saisl vcp, num sails cuts, mrle vcp, num mrle cuts, sparec, mpda vcp, base tilt vcp, num base tilts
    (ns.uint8,'reserved'), # reserved
    name='t11_hdr',padding=1,endian='>')

def create_t11_msg(data):
    els = data[32:34]
    print(els)
    return t11_msg.create()

t11_msg = ns.nstruct(
    (msg_header,'header'),
    (t11_hdr,'t11_header'),
    (t11_elev[0],'elev_data'), # elevation data
    name='t11_msg',padding=1,endian='>') # for some reason, using base=msg_base does not work here. It makes it think that is the actual raw data

In [1231]:
# message creator
def create_msg(data):
    r"""
    Create a message of the correct type from the data
    :param data: raw data
    """
    # TODO: Messages need to be merged before they can be parsed
    level = log.level
    log.setLevel(0)
    # tpyes to care about right now: {15: 5, 0: 122, 18: 4, 3: 1, 5: 1, 2: 3, 31: 10898}
    type = int.from_bytes(data[15:16], byteorder='big')
    message = None
    msg = msg_base.create(data)
    if(type==0):
        message = msg
    elif(type==31): # buggy!
        message = msg_31_raw.create(data) # TODO: need to parse the data at the pointers
    elif(type==15):
        message = msg_15.create(data)
    elif(type==2):
        message = msg_2.create(data)
    elif(type==5 or type==7):
        message = t11_msg.create(data) # TODO: need to parse number of elevations
    elif(type==18):
        # this type is weird - the reserved padding doesn't appear to be empty? The spec would make the segments useless, but the data is there. It doesn't line up. TODO: look into this!
        t = open(f"./Samples/test{msg.header.segment}.hex","wb")
        t.write(data)
        t.close()
        if(msg.header.segment == 1):
            message = msg18_1.create(data)
        elif(msg.header.segment == 2):
            message = msg18_2.create(data)
        elif(msg.header.segment == 3):
            message = msg18_3.create(data)
        elif(msg.header.segment == 4):
            message = msg18_4.create(data)
            d = ns.dump(message,typeinfo='key')
            for k,v in d['<msg18_4>']['data']['<t15_rda_adapt_data_4>'].items():
                print(f"{k}: {v}")
            log.warning(f"Segment 4: {len(message.data.padding)}")
        elif(msg.header.segment == 5):
            log.error("Segment 5 not implemented")
            message = msg18_5.create(data)
        else:
            log.warning("Unknown segment")
            message = msg
        
        
    else:
        message = msg#msg_base.create(data)

    if message is None:
        log.error(f"Message type {type} not implemented")
    log.setLevel(level)
    return message

In [1232]:
# log.setLevel(logging.FATAL-1)
record_count = len(fi.records)
print(f"Number of records: {record_count}")

record = 0
records = RECORD_HOLDER.create('')



for cr in fi.records:
    if(cr.ctrl_word < 0):
        break
    data = bz2.decompress(cr.data[:cr.ctrl_word])
    if data == b'':
        log.error(f"Error decompressing data. CW: {cr.ctrl_word}")
        break


    position = 0 # position in the data
    i = 0 # message counter
    ldm = LDM_UR.create('')
    while(position<len(data)):
        msg_size = int.from_bytes(data[position+12:position+14]) # get size in halfwords
        if(msg_size > 65535):
            log.error(f"Error: Message size too large: {msg_size}")
            raise Exception(f"Stop {msg_size}")
            break
        elif(msg_size<9):
            if (msg_size != 0):
                log.warning(f"Error?: Message size too small: {msg_size}")
                raise Exception(f"Stop {record}.{i}")
            else:
                log.info(f"Message {i} is empty")
                # break
        else:
            pass

        if(record == 0):
            eom = 2432*(i+1)-12 # not really needed but..
        else:
            eom = position+msg_size*2

        if(i==127):
            print(f"size: {msg_size} - {position} - {eom}")

        dat = data[position:eom]
        if dat[0:12] != b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00':
            log.error(f"Error: Prepad {i} is not 0: {dat[0:12]}")
        elif (dat[12:14] == b'\x00\x00' or dat[12:14] == b''):
            log.info(f"in {i}: Product is 0s: {dat[0:28]}")
        else:
            pass
        try:
            a_record_that_exists = create_msg(dat)
            ldm.msg.append(a_record_that_exists)
        except Exception as e:
            log.error(f"Error {i}.{record}: {e} -  {dat[0:28]}")
            break
        position=eom+12
        i+=1
    records.ldm.append(ldm)
    record+=1
    # log.fatal(f"going to next record {record}")



Segment 4: 1220


Number of records: 94
size: 1208 - 308864 - 311284
spares_4844_b: [0, 0, 0, 0, 0, 0, 0, 0, 64, 0, 0, 0]
spares_s: [2, 0, 0, 0, 224, 100, 239, 139, 216, 20, 0, 0, 255, 255, 255, 255, 254, 127, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 253, 255, 255, 255, 255, 255, 255, 160, 32, 239, 139, 216, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 187, 207, 185, 139, 216, 20, 0, 0, 11, 0, 0, 0, 48, 0, 0, 0, 32, 167, 217, 17, 254, 127, 0, 0, 220, 136, 70, 0, 0, 0, 0, 0, 177, 160, 217, 17, 254, 127, 0, 0, 160, 220, 240, 139, 216, 20, 0, 0, 16, 165, 217, 17, 254, 127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 112, 70, 0, 0, 0, 0, 0, 32, 0, 0, 0, 48, 0, 0, 0, 32, 167, 217, 17, 254, 127, 0, 0, 96, 166, 217, 17, 254, 127, 0, 0, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 8, 164, 217, 17, 254, 127, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 122, 112, 70, 0, 0, 0, 0, 0, 104, 52, 9, 1, 0, 0, 0, 0, 140, 38, 104, 0, 0, 0, 0, 0, 2

Error 104.46: Cannot parse struct: data is corrupted. -  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00D\x08\x02\x08iM\x99\x04\x0c\x86\x11\x00\x01\x00\x01'
Error 13.47: Cannot parse struct: data is corrupted. -  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00D\x08\x02\x08\x87M\x99\x04\x0c\x8a\x01\x00\x01\x00\x01'


In [1233]:
def checkMsg(msg,i,j):
    warns = 0
    infos = 0
    if (msg is None):
        log.error(f"Error: Message {i}.{j} is None")
        return -1
    if msg.header.message_type > 33:
        log.warning(f"Message {i}.{j} type {msg.header.message_type} is not valid!")
        warns+=1
    if msg.header.message_type == 0:
        log.info(f"Message {i}.{j} has type 0. Likely a type 13 legacy area.")
        infos+=1
    if msg.header.milliseconds >= 86400000:
        log.warning(f"Message {i}.{j} time is invalid: {msg.header.milliseconds}")
        warns+=1
    if msg.header.segments > 1 or msg.header.segment > 1:
        log.warning(f"Message {i}.{j} has segments: {msg.header.segment} of {msg.header.segments}")
        warns+=1
    if msg.header.segments == 0 or msg.header.segment == 0:
        log.info(f"Message {i}.{j} has no segments: {msg.header.segment} of {msg.header.segments}")
        infos+=1
    if msg.header.date > 65535:
        log.warning(f"Message {i}.{j} date is invalid: {msg.header.date}")
        warns+=1
    if msg.header.sequence > 65535:
        log.warning(f"Message {i}.{j} sequence is invalid: {msg.header.sequence}")
        warns+=1
    if msg.header.sequence == 0:
        log.info(f"Message {i}.{j} has sequence number 0")
        infos+=1
    if msg.header.channel > 10:
        log.warning(f"Message {i}.{j} redundant channel is invalid: {msg.header.channel}")
        warns+=1
    if msg.header.message_size > 65535:
        log.warning(f"Message {i}.{j} size is invalid: {msg.header.message_size}")
        warns+=1
    if msg.header.prepad[0] != 0 or msg.header.prepad[1] != 0 or msg.header.prepad[2] != 0:
        log.warning(f"Message {i}.{j} Prepad is not 0: {msg.header.prepad}")
        warns+=1
    if(warns > 0):
        log.warning(f"...Message {i}.{j} - W:{warns} - {msg.header.message_type} has sequence number {msg.header.sequence} and is {msg.header.message_size} halfwords")
    type = msg.header.message_type
    if(type is None):
        log.error(f"Error: Message {i}.{j} type is None")
    return type
    

In [1234]:

log.setLevel(1000)
prods = {}
totals = 0
for i, ldm in enumerate(records.ldm):
    for j, msg in enumerate(ldm.msg):
        try:
            msgtype = checkMsg(msg,i,j)
            if(msgtype == 31 and i==1 and j==5):
                dump = ns.dump(msg,humanread=True,typeinfo='key')
                # print(dump)
                # for k,v in dump['<msg_2>'].items():
                #     print(f"{k}: {v}")
            prods[msgtype] = prods.get(msgtype,0)+1
        except Exception as e:
            print(e)
        
        totals+=1
print(prods)
print(totals)
# 3.2.4.17

{15: 5, 0: 122, 18: 4, 3: 1, 5: 1, 2: 1, 31: 10917}
11051
